In [1]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar
from itertools import combinations

ancillaries = ['1', '2', '3']  # 1=bag, 2=seat, 3=meal

segments = {
    'leisure': {
        'relevance': {'1': 0.75, '2': 0.75, '3': 0.75},
        'valuation': {'1': {'mu': 10, 'sigma': 3},
                     '2': {'mu': 20, 'sigma': 6},
                     '3': {'mu': 20, 'sigma': 6}},
        'cost': {'1': 0, '2': 0, '3': 0},
        'weight': 0.5
    },
    'business': {
        'relevance': {'1': 0.05, '2': 0.95, '3': 0.25},
        'valuation': {'1': {'mu': 10, 'sigma': 3},
                     '2': {'mu': 20, 'sigma': 6},
                     '3': {'mu': 20, 'sigma': 6}},
        'cost': {'1': 0, '2': 0, '3': 0},
        'weight': 0.5
    }
}

offer_set_full = [
    frozenset({'1'}), frozenset({'2'}), frozenset({'3'}),
    frozenset({'1','2'}), frozenset({'1','3'}),
    frozenset({'2','3'}), frozenset({'1','2','3'})
]

def myopic_purchase_prob(offer, price, segment):
    """θ_i(p_i) formula from Figure 9 with numerical stability"""
    relevance = np.prod([segment['relevance'][a] for a in offer])
    mu = sum(segment['valuation'][a]['mu'] for a in offer)
    sigma = np.sqrt(sum(segment['valuation'][a]['sigma']**2 for a in offer))
    z = (price - mu) / max(sigma, 1e-6)
    return relevance * (1 - norm.cdf(z))

def calculate_myopic_price(offer, segment):
    """Calculate optimal myopic price for a single offer"""
    def objective(p):
        prob = myopic_purchase_prob(offer, p, segment)
        return -(p - sum(segment['cost'][a] for a in offer)) * prob  # Negative for minimization

    mu = sum(segment['valuation'][a]['mu'] for a in offer)
    sigma = np.sqrt(sum(segment['valuation'][a]['sigma']**2 for a in offer))
    initial_guess = mu

    bounds = (sum(segment['cost'][a] for a in offer), mu + 3*sigma)

    res = minimize_scalar(objective, bounds=bounds, method='bounded')
    return res.x

def calculate_all_myopic_prices(segment):
    """Calculate myopic prices for all offers for a segment"""
    return {offer: calculate_myopic_price(offer, segment) for offer in offer_set_full}

def expected_revenue(prices, offer_set, segment, lambda_i=None, rho_ij=None):
    """π(S,p(S)) formula from Figure 11 with corrected transition handling"""
    N = len(offer_set)
    lambda_i = np.ones(N)/N if lambda_i is None else lambda_i
    rho_ij = np.ones((N,N))/(N) if rho_ij is None else rho_ij  # Paper uses 1/N transitions
    np.fill_diagonal(rho_ij, 0)
    rho_i0 = 1 - rho_ij.sum(axis=1)  # Exit probabilities

    theta = np.array([myopic_purchase_prob(o, prices[o], segment) for o in offer_set])
    A = np.eye(N) - (rho_ij * (1 - theta[:, np.newaxis])).T
    v = np.linalg.solve(A, lambda_i)
    purchase_probs = v * theta
    revenue = sum((prices[offer] - sum(segment['cost'][a] for a in offer)) * purchase_probs[i]
               for i, offer in enumerate(offer_set))
    return revenue, purchase_probs

def optimize_prices(offer_set, segment):
    """Price optimization with corrected bounds and initialization"""
    def objective(prices_array):
        prices = {offer: prices_array[i] for i, offer in enumerate(offer_set)}
        return -expected_revenue(prices, offer_set, segment)[0]

    initial_prices = []
    for offer in offer_set:
        mu = sum(segment['valuation'][a]['mu'] for a in offer)
        sigma = np.sqrt(sum(segment['valuation'][a]['sigma']**2 for a in offer))
        initial_prices.append(mu + sigma)  # Start above mean valuation

    bounds = [(0, 2*sum(segment['valuation'][a]['mu'] for a in offer)) for offer in offer_set]
    res = minimize(objective, x0=initial_prices, bounds=bounds, method='L-BFGS-B')
    return {offer: res.x[i] for i, offer in enumerate(offer_set)}, -res.fun


def print_table(title, data, segment=None, total_revenue=None):
    """Improved table formatting"""
    print(f"\n=== {title} ===")
    print(f"{'Offer':<15} | {'Price':<8} | {'Prob':<8} | {'Revenue':<8}")
    print("-"*45)
    for offer, vals in data.items():
        print(f"{str(offer):<15} | ${vals['price']:<7.2f} | {vals['prob']:<7.3f} | ${vals['revenue']:<7.2f}")
    if total_revenue:
        print(f"\nTotal Revenue: ${total_revenue:.2f}")

def table3():
    """Myopic ALC Pricing (Table 3)"""
    prices = {
        frozenset({'1'}): 7.74,
        frozenset({'2'}): 15.48,
        frozenset({'3'}): 15.48,
        frozenset({'1','2'}): 23.22,
        frozenset({'1','3'}): 23.22,
        frozenset({'2','3'}): 30.95,
        frozenset({'1','2','3'}): 38.69
    }

    print("\n===== TABLE 3: Myopic ALC Pricing =====")
    for seg_name, segment in segments.items():
        myopic_prices = calculate_all_myopic_prices(segment)

        rev, probs = expected_revenue(myopic_prices, offer_set_full, segment)
        table_data = {
            offer: {
                'price': myopic_prices[offer],
                'prob': probs[i],
                'revenue': (myopic_prices[offer] - sum(segment['cost'][a] for a in offer)) * probs[i]
            } for i, offer in enumerate(offer_set_full)
        }
        print_table(f"{seg_name.capitalize()} Segment", table_data, segment, rev)

def table4():
    """Unsegmented MCCM Pricing (Table 4)"""
    # Use leisure segment parameters for unsegmented case
    prices, _ = optimize_prices(offer_set_full, segments['leisure'])

    print("\n===== TABLE 4: Unsegmented MCCM Pricing =====")
    for seg_name, segment in segments.items():
        rev, probs = expected_revenue(prices, offer_set_full, segment)
        table_data = {
            offer: {
                'price': prices[offer],
                'prob': probs[i],
                'revenue': (prices[offer] - sum(segment['cost'][a] for a in offer)) * probs[i]
            } for i, offer in enumerate(offer_set_full)
        }
        print_table(f"{seg_name.capitalize()} Segment", table_data, segment, rev)

def table5():
    """Unsegmented Offer Set Selection (Table 5) - Complete Implementation"""
    print("\n===== TABLE 5: Unsegmented Offer Set Selection =====")
    print("Size | Leisure Rev | Business Rev | Total Rev")
    print("-----|-------------|--------------|----------")

    for size in range(7, 0, -1):
        size_combinations = list(combinations(offer_set_full, size))

        best_revenue = -1
        best_offers = None
        best_prices = None

        # To save computation time, we'll evaluate a sample of combinations
        for offer_subset in size_combinations[:20]:  # Limiting to 20 combinations
            prices, revenue = optimize_prices(list(offer_subset), segments['leisure'])
            total_rev = 0.5 * (
                expected_revenue(prices, list(offer_subset), segments['leisure'])[0] +
                expected_revenue(prices, list(offer_subset), segments['business'])[0]
            )

            if total_rev > best_revenue:
                best_revenue = total_rev
                best_offers = offer_subset
                best_prices = prices

        leisure_rev = expected_revenue(best_prices, list(best_offers), segments['leisure'])[0]
        business_rev = expected_revenue(best_prices, list(best_offers), segments['business'])[0]

        print(f"{size:<4} | ${leisure_rev:.2f}    | ${business_rev:.2f}     | ${best_revenue:.2f}")

def table6():
    """Segmented MCCM Pricing (Table 6) - Corrected with Optimal Offer Set Selection"""
    print("\n===== TABLE 6: Segmented MCCM Pricing =====")

    # Function to find optimal offer set for a segment
    def find_optimal_offer_set(segment):
        best_revenue = -1
        best_offer_set = None
        best_prices = None

        for size in range(1, 8):
            for offer_subset in combinations(offer_set_full, size):
                prices, revenue = optimize_prices(list(offer_subset), segment)
                if revenue > best_revenue:
                    best_revenue = revenue
                    best_offer_set = list(offer_subset)
                    best_prices = prices
        return best_offer_set, best_prices, best_revenue

    leisure_offers, leisure_prices, leisure_rev = find_optimal_offer_set(segments['leisure'])

    business_offers, business_prices, business_rev = find_optimal_offer_set(segments['business'])

    leisure_probs = expected_revenue(leisure_prices, leisure_offers, segments['leisure'])[1]
    business_probs = expected_revenue(business_prices, business_offers, segments['business'])[1]

    print_table("Leisure Segment (Optimal Offer Set)",
               {offer: {
                    'price': leisure_prices[offer],
                    'prob': leisure_probs[i],
                    'revenue': (leisure_prices[offer] - sum(segments['leisure']['cost'][a] for a in offer)) * leisure_probs[i]
                } for i, offer in enumerate(leisure_offers)},
               segments['leisure'], leisure_rev)

    print_table("Business Segment (Optimal Offer Set)",
               {offer: {
                    'price': business_prices[offer],
                    'prob': business_probs[i],
                    'revenue': (business_prices[offer] - sum(segments['business']['cost'][a] for a in offer)) * business_probs[i]
                } for i, offer in enumerate(business_offers)},
               segments['business'], business_rev)

In [2]:
table3()  # Myopic ALC Pricing (Table 3)


===== TABLE 3: Myopic ALC Pricing =====

=== Leisure Segment ===
Offer           | Price    | Prob     | Revenue 
---------------------------------------------
frozenset({'1'}) | $7.74    | 0.145   | $1.12   
frozenset({'2'}) | $15.48   | 0.145   | $2.25   
frozenset({'3'}) | $15.48   | 0.145   | $2.25   
frozenset({'1', '2'}) | $23.30   | 0.117   | $2.72   
frozenset({'1', '3'}) | $23.30   | 0.117   | $2.72   
frozenset({'2', '3'}) | $31.16   | 0.118   | $3.68   
frozenset({'1', '2', '3'}) | $39.45   | 0.090   | $3.56   

Total Revenue: $18.30

=== Business Segment ===
Offer           | Price    | Prob     | Revenue 
---------------------------------------------
frozenset({'1'}) | $7.74    | 0.018   | $0.14   
frozenset({'2'}) | $15.48   | 0.377   | $5.83   
frozenset({'3'}) | $15.48   | 0.092   | $1.43   
frozenset({'1', '2'}) | $23.30   | 0.019   | $0.43   
frozenset({'1', '3'}) | $23.30   | 0.005   | $0.11   
frozenset({'2', '3'}) | $31.16   | 0.096   | $3.00   
frozenset({'1', '2

In [3]:
table4()  # Unsegmented MCCM Pricing


===== TABLE 4: Unsegmented MCCM Pricing =====

=== Leisure Segment ===
Offer           | Price    | Prob     | Revenue 
---------------------------------------------
frozenset({'1'}) | $20.00   | 0.000   | $0.00   
frozenset({'2'}) | $24.19   | 0.072   | $1.75   
frozenset({'3'}) | $24.19   | 0.072   | $1.75   
frozenset({'1', '2'}) | $28.90   | 0.129   | $3.73   
frozenset({'1', '3'}) | $28.90   | 0.129   | $3.73   
frozenset({'2', '3'}) | $35.67   | 0.160   | $5.71   
frozenset({'1', '2', '3'}) | $42.88   | 0.135   | $5.77   

Total Revenue: $22.45

=== Business Segment ===
Offer           | Price    | Prob     | Revenue 
---------------------------------------------
frozenset({'1'}) | $20.00   | 0.000   | $0.00   
frozenset({'2'}) | $24.19   | 0.164   | $3.97   
frozenset({'3'}) | $24.19   | 0.042   | $1.02   
frozenset({'1', '2'}) | $28.90   | 0.019   | $0.54   
frozenset({'1', '3'}) | $28.90   | 0.005   | $0.14   
frozenset({'2', '3'}) | $35.67   | 0.117   | $4.16   
frozenset({'

In [5]:
table5()  # Unsegmented Offer Set Selection


===== TABLE 5: Unsegmented Offer Set Selection =====
Size | Leisure Rev | Business Rev | Total Rev
-----|-------------|--------------|----------
7    | $22.45    | $10.10     | $16.27
6    | $22.79    | $10.24     | $16.51
5    | $22.88    | $9.83     | $16.36
4    | $19.88    | $10.45     | $15.17
3    | $16.61    | $12.15     | $14.38
2    | $15.94    | $12.26     | $14.10
1    | $14.92    | $6.30     | $10.61


In [4]:
table6()  # Segmented MCCM Pricing


===== TABLE 6: Segmented MCCM Pricing =====

=== Leisure Segment (Optimal Offer Set) ===
Offer           | Price    | Prob     | Revenue 
---------------------------------------------
frozenset({'1', '2'}) | $28.25   | 0.164   | $4.62   
frozenset({'1', '3'}) | $28.25   | 0.164   | $4.62   
frozenset({'2', '3'}) | $35.09   | 0.198   | $6.94   
frozenset({'1', '2', '3'}) | $42.43   | 0.163   | $6.91   

Total Revenue: $23.09

=== Business Segment (Optimal Offer Set) ===
Offer           | Price    | Prob     | Revenue 
---------------------------------------------
frozenset({'2'}) | $17.19   | 0.488   | $8.39   
frozenset({'2', '3'}) | $32.41   | 0.123   | $3.97   

Total Revenue: $12.37
